In [1]:
%pip install ollama
%pip install pdfplumber


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import ollama
import pdfplumber

def read_insurance_policy(file_name):
    with pdfplumber.open(file_name) as pdf:  
        first_page = pdf.pages[0].filter(lambda obj: not (obj["object_type"] == "char" and obj["size"] < 10))
        #print(f"first_page_words: {first_page.extract_words}")
        #print(f"first_page_text_lines: {first_page.extract_text_lines}")
        #print(f"first_page_text: {first_page.extract_text}")
        #print(f"first_page_text_simple: {first_page.extract_text_simple}")
        #print(f"first_page_tables: {first_page.extract_tables}") 
        second_page = pdf.pages[1]
        #print(f"second_page: {second_page} ")
        policy_text = first_page.extract_text()
        policy_text += "\n" + second_page.extract_text()

    #print(f"policy_text: {policy_text}")
    prompt = f"""You are an expert insurance policy analyzer. Extract the following details from the insurance policy text provided below:
    1. Name of the Policyholder
    2. Policy Number
    3. Coverage Summary
    4. Premium Amount
    5. Start Date
    6. End Date
    
    Policy Text:
    {policy_text}
    
    Please provide the information in a structured format."""
    
    # Use Ollama to analyze the policy text
    #response = ollama.generate(model="llama3", prompt=prompt)
    
    # Use GPT5-mini to analyze the policy text
    #response = ollama.generate(model="gpt-oss:20b", prompt=prompt)
    response = ollama.generate(model="gpt-oss:120b", prompt=prompt)  
    # For now, return extracted data structure
    return {
        "Name": policy_text.find("Name"),
        "Policy Number": policy_text.find("Policy Number"),
        "Coverage Summary": policy_text.find("Coverage Summary"),
        "Premium Amount": policy_text.find("Premium Amount"),
        "Start Date": policy_text.find("Start Date"),
        "End Date": policy_text.find("End Date")
    }


In [4]:
import json
policy_details = read_insurance_policy("./data/Progressive ID Card.pdf")
print(f"policy_details: {policy_details}")
json_str = json.dumps(policy_details, indent=4)
print(f"policy_details JSON: {json_str}")

try:
    #with open("policy_details_llama.json", "w") as json_file:
    with open("policy_details_gpt.json", "w") as json_file:
        json_file.write(json_str)
    print("Policy details saved to policy_details.json")
except Exception as e: 
    print(f"Error saving policy details: {e}")


ResponseError: model 'gpt-oss:120b' not found (status code: 404)